In [12]:
import pandas as pd
import numpy as np

In [13]:
def get_percentile(df):
    df = df.fillna(np.inf).drop("Date", axis=1)
    percentile_month  = df.quantile([0.05, 0.25, 0.50])/ 30.25
    return percentile_month.T

Fractile Analysis

In [14]:
fractile_df = pd.read_excel('results/FractilePortfolio.xlsx')
pure_df =  pd.read_excel('results/PureFactorPortfolio.xlsx')

In [15]:
get_percentile(fractile_df)

c:\Users\bapdu\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Users\bapdu\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4658: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t >= 0.5,


,0.05,0.25,0.50
Momentum,3.014876,4.619835,5.900826
Value,3.900826,20.826446,NaN
Quality,20.102479,NaN,NaN
Low Volatility,12.294215,21.834711,29.652893
Market,12.476033,22.636364,NaN


In [17]:
get_percentile(pure_df)

,0.05,0.25,0.50
Momentum,3.504132,5.421488,6.297521
Value,1.464463,6.966942,20.000000
Quality,6.016529,12.033058,25.107438
Low Volatility,7.095868,11.223140,18.115702
Market,7.117355,10.867769,17.867769
